# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking your current working directory
print('Current working directory:', os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # ignore the ipynb_checkpoints files
    if '/.ipynb_checkpoints' in root:
        continue
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

Current working directory: /home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)
            
# uncomment the code below if you would like to get total number of rows 
print('Total number of rows',len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

Total number of rows 8056


In [4]:
# Check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('Number of rows in csv file:',sum(1 for line in f))

Number of rows in csv file: 6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Connect to Database

#### Creating a Cluster

In [5]:
# Connection to a Cassandra instance local machine (127.0.0.1) 
try:
    from cassandra.cluster import Cluster
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace
try:
    session.execute('''
        CREATE KEYSPACE IF NOT EXISTS project2
        WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    ''')
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('project2')

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Tabel session_song format:  
```
session_song (
    session_id int
    item_in_session int
    artist text
    song_title text
    song_length float
    PRIMARY KEY (session_id, item_in_session)
)
```
Composite Partition Key: session_id, item_in_session

In [8]:
## Query 1:
## Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS session_song"
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_length float, \
                  PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# Insert data from event_datafile_new.csv into session_song table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_song (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
# Verify session_song table
query = 'SELECT session_id FROM session_song'

try:
    rows = session.execute(query)
    # print first 5 rows in table
    for row in rows:
        print(row)
except Exception as e:
    print(e)

In [11]:
# Answer Query 1
query = 'SELECT artist, song_title, song_length FROM session_song WHERE session_id=338 AND item_in_session=4'

try:
    rows = session.execute(query)
    for row in rows:
        print(row.artist, row.song_title, row.song_length)
except Exception as e:
    print(e)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Tabel session_user format:

```
session_user (
    session_id int
    user_id int
    item_in_session int
    artist text
    song_title text
    first_name text
    last_name text
    PRIMARY KEY ((session_id, user_id), item_in_session)
)
```
Composite Partition Key: session_id, user_id
Clustering Key: item_in_session

In [11]:
## Query 2: 
## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS session_user"
query = query + "(session_id int, user_id int, item_in_session int, artist text, song_title text, first_name text, last_name text, \
                  PRIMARY KEY ((session_id, user_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# Insert data from event_datafile_new.csv into session_user table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_user (session_id, user_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
# Verify session_user table
query = 'SELECT * FROM session_user'

try:
    rows = session.execute(query)
    # print first 5 rows in table
    for row in rows[:5]:
        print(row)
except Exception as e:
    print(e)

Row(session_id=1049, user_id=73, item_in_session=0, artist='Manowar', first_name='Jacob', last_name='Klein', song_title='Shell Shock')
Row(session_id=1049, user_id=73, item_in_session=1, artist='Morcheeba', first_name='Jacob', last_name='Klein', song_title='Women Lose Weight (Feat: Slick Rick)')
Row(session_id=1049, user_id=73, item_in_session=2, artist='Maroon 5', first_name='Jacob', last_name='Klein', song_title="Won't Go Home Without You")
Row(session_id=1049, user_id=73, item_in_session=3, artist='Train', first_name='Jacob', last_name='Klein', song_title='Hey_ Soul Sister')
Row(session_id=1049, user_id=73, item_in_session=4, artist='LMFAO', first_name='Jacob', last_name='Klein', song_title="I'm In Miami Bitch")


In [15]:
# Answer Query 2
query = 'SELECT artist, song_title, first_name, last_name FROM session_user WHERE session_id=182 AND user_id=10'
try:
    rows = session.execute(query)
    for row in rows:
        print(row.artist, row.song_title, row.first_name, row.last_name)
except Exception as e:
    print(e)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Tabel user_song format:

```
session_user (
    song_title text
    user_id int
    first_name text
    last_name text
    PRIMARY KEY (song_title, user_id)
)
```
Composite Partition Key: song_title, user_id

In [16]:
## Query 3: 
## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS user_song"
query = query + "(song_title text, user_id int, first_name text, last_name text, \
                  PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [17]:
# Insert data from event_datafile_new.csv into user_song table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (song_title, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [18]:
# Verify session_user table
query = 'SELECT * FROM user_song'

try:
    rows = session.execute(query)
    # print first 5 rows in table
    for row in rows[:5]:
        print(row)
except Exception as e:
    print(e)

Row(song_title="Wonder What's Next", user_id=49, first_name='Chloe', last_name='Cuevas')
Row(song_title="In The Dragon's Den", user_id=49, first_name='Chloe', last_name='Cuevas')
Row(song_title='Too Tough (1994 Digital Remaster)', user_id=44, first_name='Aleena', last_name='Kirby')
Row(song_title='Rio De Janeiro Blue (Album Version)', user_id=49, first_name='Chloe', last_name='Cuevas')
Row(song_title='My Place', user_id=15, first_name='Lily', last_name='Koch')


In [19]:
# Answer Query 3
query = "SELECT first_name, last_name FROM user_song WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
    for row in rows:
        print(row.first_name, row.last_name)
except Exception as e:
    print(e)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [20]:
#Drop the table before closing out the sessions
session_song_tabel_drop = 'DROP TABLE IF EXISTS session_song'
session_user_tabel_drop = 'DROP TABLE IF EXISTS session_user'
user_song_tabel_drop = 'DROP TABLE IF EXISTS user_song'

drop_table_queries = [session_song_tabel_drop,session_user_tabel_drop,user_song_tabel_drop]

for query in drop_table_queries:
    session.execute(query)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()

## OPTIONAL: Question for the reviewer
 
If you have any question about the starter code or your own implementation, please add it in the cell below. 

For example, if you want to know why a piece of code is written the way it is, or its function, or alternative ways of implementing the same functionality, or if you want to get feedback on a specific part of your code or get feedback on things you tried but did not work.

Please keep your questions succinct and clear to help the reviewer answer them satisfactorily. 

> **_Your question_**

I spend a lot of time on naming tables, if you have any document on table naming standards, please let me know.